# Random Forest

In [2]:
# !pip install imblearn

import os
import pandas as pd
import numpy as np

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

# Function for creating model pipelines - sklearn
from sklearn.pipeline import make_pipeline

# Function for creating model pipelines - imblearn
from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE
# Import classifier
from sklearn.linear_model import LogisticRegression
import datetime as dt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

## Read the data from a parquet file


In [ ]:
df = pd.read_parquet('df_age_18.parquet')

## Display counts and percentages of the target variable

In [ ]:
# Display counts and percentages
value_counts = df['Y'].value_counts()
percentages = df['Y'].value_counts(normalize=True) * 100

result_df = pd.DataFrame({'Count': value_counts, 'Percentage': percentages})
print(result_df)

## Drop unnecessary columns from the dataframe


In [ ]:
df.drop(columns=['#','EBANK_FIRST_OPEN_DATE','CIF_OPEN_DATE',
 'MB2_FIRST_OPEN_DATE',
 'SYM_RUN_DATE'],inplace=True)

## Check for missing values and fill them with 0

In [ ]:
df.isnull().sum()

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
df.isnull().sum()

## Select only numerical features for correlation analysis

In [ ]:
df_corr = df[df.select_dtypes(include='number').columns.tolist()]

## Function to select highly correlated features


In [ ]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

## Find highly correlated features and drop them from the dataframe

In [ ]:
corr_features = correlation(df_corr, 0.7)
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
df_filtered = df.drop(corr_features,axis=1)

In [ ]:
df_filtered.to_parquet('df_filtered_corr.parquet')

In [ ]:
df_filtered = pd.read_parquet('df_filtered_corr.parquet')

## Separate dataframe into separate object

In [ ]:
#Separate dataframe into separate object

# Object for target variable
y = df_filtered.Y

# object for input features
X = df_filtered.drop(['CLIENT_NO','Y'], axis=1)

# display shapes of X and y
print(X.shape, y.shape)

## Get the numerical and categorical columns for preprocessing


In [ ]:
# List numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

In [ ]:
# List categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

## Split the data into train and test sets


In [ ]:
random_state = 10

# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=random_state,
                                                    stratify=df_filtered.Y)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
X_train.info()

In [ ]:
num_features = []

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features)

In [ ]:
cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features)

In [ ]:
# Define column transformer
# Need to be numeric not string to specify columns name
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(sparse=False), cat_features)
)
preprocess

## Fit the preprocessing pipeline on the train data


In [ ]:
# Fit preprocessing pipeline on train data
preprocess.fit(X_train)

## Apply the preprocessing pipeline to train and test data


In [ ]:
# Apply preprocessing pipeline to train and test data
X_train_preprocessed = preprocess.transform(X_train)
X_test_preprocessed = preprocess.transform(X_test)

## Apply SMOTE to handle class imbalance in the train data


In [ ]:
# Define SMOTE object
smote = SMOTE(sampling_strategy='auto', random_state=random_state)

In [ ]:
# Fit SMOTE on preprocessed train data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)

In [ ]:
len(X_train_resampled)

In [ ]:
160645

In [ ]:
pd.DataFrame(X_train_resampled).to_parquet("X_train_resampled.parquet",index=False)
pd.DataFrame(y_train_resampled).to_parquet("y_train_resampled.parquet",index=False)
pd.DataFrame(X_test_preprocessed).to_parquet("X_test_preprocessed.parquet",index=False)
pd.DataFrame(y_test).to_parquet("y_test.parquet",index=False)

## Define the KNN classifier and set hyperparameters for grid search


In [4]:
X_train_resampled = pd.read_parquet('/content/drive/MyDrive/GGCLB/X_train_resampled.parquet')
y_train_resampled = pd.read_parquet('/content/drive/MyDrive/GGCLB/y_train_resampled.parquet')
X_test_preprocessed = pd.read_parquet('/content/drive/MyDrive/GGCLB/X_test_preprocessed.parquet')
y_test = pd.read_parquet('/content/drive/MyDrive/GGCLB/y_test.parquet')

In [5]:
# Define RF classifier
rfc = RandomForestClassifier(random_state=10)

In [7]:
param_grid = {
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,6]
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, scoring = 'accuracy', verbose = 3)
CV_rfc.fit(X_train_resampled, y_train_resampled)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 1/5] END ....max_depth=4, max_features=sqrt;, score=0.938 total time= 1.4min


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 2/5] END ....max_depth=4, max_features=sqrt;, score=0.938 total time= 1.3min


...

[CV 4/5] END ....max_depth=6, max_features=log2;, score=0.942 total time= 1.2min


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV 5/5] END ....max_depth=6, max_features=log2;, score=0.942 total time= 1.2min


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=10),
             param_grid={'max_depth': [4, 6], 'max_features': ['sqrt', 'log2']},
             scoring='accuracy', verbose=3)

## Print the best hyperparameters found by grid search


In [8]:
print(CV_rfc.best_estimator_)

RandomForestClassifier(max_depth=6, random_state=10)


In [9]:
print(CV_rfc.best_params_)

{'max_depth': 6, 'max_features': 'sqrt'}


In [12]:
predictions_train = CV_rfc.predict(X_train_resampled)
print(classification_report(y_train_resampled, predictions_train))


              precision    recall  f1-score   support

           0       0.94      0.94      0.94    803226
           1       0.94      0.94      0.94    803226

    accuracy                           0.94   1606452
   macro avg       0.94      0.94      0.94   1606452
weighted avg       0.94      0.94      0.94   1606452



In [10]:
predictions_test = CV_rfc.predict(X_test_preprocessed)
print(classification_report(y_test, predictions_test))


              precision    recall  f1-score   support

           0       0.89      0.94      0.91    183129
           1       0.97      0.94      0.95    344240

    accuracy                           0.94    527369
   macro avg       0.93      0.94      0.93    527369
weighted avg       0.94      0.94      0.94    527369



In [13]:
import joblib
filename = '/RF.sav'
joblib.dump(CV_rfc, filename)

['/content/drive/MyDrive/GGCLB/RF.sav']